## Analiza danych
### Hubert Kłosowski 242424
### Kamil Małecki 242464

### Potrzebne importy

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import datetime as dt
from sklearn.preprocessing import PolynomialFeatures
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller

### Wczytanie danych

In [ ]:
def get_data():
    os.chdir('..')
    os.chdir('Zadanie1\\czesc4')
    read = []
    for filename in os.listdir():
        df = pd.read_csv(filename)
        if 'date' in df.columns:
            df['date'] = pd.to_datetime(df['date'])
        read.append(df)
    os.chdir('..')
    os.chdir('..')
    os.chdir('Zadanie2')
    return read

data = get_data()
part6 = data[5]

## <center>Część 1</center>
### Wyznacz średnią kroczącą i odchylenie standardowe kroczące (np. 7-dniowe), aby zobaczyć, jak te zjawiska zmieniają się w czasie. Obliczenia wykonaj dla:


### 6 krajów wydających najwięcej na służbę zdrowia

In [ ]:
countries = part6[['country_name', 'health_expenditure_usd']].drop_duplicates().sort_values(by='health_expenditure_usd', ascending=False)[:6]
countries = countries.reset_index(drop=True)
countries.drop(['health_expenditure_usd'], inplace=True, axis=1)
countries = countries['country_name'].tolist()

### Średnia ruchoma i odchylenie standardowe

In [ ]:
def calculate_moving_avg_std(column, country_name):
    mean_data = part6.loc[part6['country_name'] == country_name, column].rolling(window=7).mean().bfill()
    std_data = part6.loc[part6['country_name'] == country_name, column].rolling(window=7).std().bfill()
    return pd.DataFrame({'data': part6.loc[part6['country_name'] == country_name, column], 'mean': mean_data, 'std': std_data}).reset_index(drop=True)

def plot_moving_avg_std(ylabel, column):
    fig, ax = plt.subplots(nrows=3, ncols=2, figsize=(20, 12))
    fig.suptitle('Średnia i ochylenie standardowe kroczące 7-dniowe', fontsize=20)
    for i, country_name in enumerate(countries):
        x_cord, y_cord = divmod(i, 2)
        moving_avg_std = calculate_moving_avg_std(column, country_name)
        x_axis = part6['date'].unique()
        sns.lineplot(y=moving_avg_std['data'], x=x_axis, ax=ax[x_cord, y_cord], label='dniowe wartości')
        sns.lineplot(y=moving_avg_std['mean'], x=x_axis, ax=ax[x_cord, y_cord], color='red', label='średnia krocząca')
        sns.lineplot(y=moving_avg_std['std'], x=x_axis, ax=ax[x_cord, y_cord], color='green', label='odchylenie standardowe kroczące')
        ax[x_cord, y_cord].set_title(country_name)
        ax[x_cord, y_cord].set_xlabel('')
        ax[x_cord, y_cord].set_ylabel('')
        if x_cord == 2:
            ax[x_cord, y_cord].set_xlabel('Date')
        if y_cord == 0:
            ax[x_cord, y_cord].set_ylabel(ylabel)
        ax[x_cord, y_cord].legend(loc='best')

### 1.1. liczby nowych zachorowań,

In [ ]:
plot_moving_avg_std('Liczba nowych zachorowań', 'daily_confirmed')

### 1.2. liczby nowych śmierci,

In [ ]:
plot_moving_avg_std('Liczba nowych śmierci', 'daily_deceased')

### 1.3. liczby nowych szczepień,

In [ ]:
plot_moving_avg_std('Liczba nowych szczepień', 'daily_persons_vaccinated')

### 1.4 liczba nowych zachorowań mężczyzn,

In [ ]:
plot_moving_avg_std('Liczba nowych zachorowań mężczyzn', 'daily_confirmed_male')

### 1.4 liczba nowych zachorowań kobiet

In [ ]:
plot_moving_avg_std('Liczba nowych zachorowań kobiet', 'daily_confirmed_female')

## <center>Część 2</center>
### Wykorzystaj do analizy trendów analizę szeregów czasowych (metoda średniej ruchomej czy inne modele autoregresyjne), która umożliwi zbadanie 5 przypadków rozważanych w części 1 poziomu 3. Przeanalizuj otrzymane wyniki.

### Sprawdzenie typu szeregu czasowego

test ocenia czy szereg czasowy jest stacjonarny lub nie stacjonarny.
Stacjonarny szereg to taki, którego właściwości nie zależą od czasu.
Natomiast te, które wykazują trend czy sezonowość nie są stacjonarne.

<p><b>Augmented Dickey-Fuller Test</b></p>
<p></p>

<p>Hipoteza zerowa (H0): Szereg czasowy nie jest stacjonarny (jego średnia i wariancja jest stała)</p>
<p>Alternatywna hipoteza (H1): Szereg czasowy jest stacjonarny</p>
    <p>p-value &gt; 0.05 -> (H0)</p>
    <p>p-value &le; 0.05 -> (H1)</p>

<ol>
    <li>wartość testu ADF - większa wartość ujemna = większe prawdopodobieństwo, że szereg jest stacjonarny</li>
    <li>p-value</li>
    <li>liczba opóźnień</li>
    <li>liczba obserwacji</li>
    <li>krytyczne wartości</li>
    <li>wartość krytyczna</li>
</ol>
<h3>ACF (Auto-Correlation Function)</h3>

Funkcja autokorelacji pokazuje jak zmienia się korelacja pomiędzy dwiema dowolnymi wartościami szeregu wraz ze zmianą ich odstępu. Jest to miara, która pomaga zrozumieć, czy istnieje jakakolwiek zależność między wartościami szeregu czasowego a ich poprzednimi wartościami.

### Dane treningowe i testowe

In [ ]:
def get_train_test(country_name, column, start, end, test_size=0.25):
    total_len = pd.to_datetime(end) - pd.to_datetime(start)
    train_range = np.ceil((1 - test_size) * total_len.days)
    end_train_date = pd.to_datetime(start) + pd.Timedelta(train_range, unit='d')
    train_data = part6.loc[(part6['country_name'] == country_name) & (part6['date'].between(start, end_train_date)), ['date', column]]
    train_data.set_index('date', inplace=True)
    train_data.index.freq = 'D'
    test_data = part6.loc[(part6['country_name'] == country_name) & (part6['date'].between(pd.to_datetime(end_train_date) + pd.Timedelta(1, unit='d'), pd.to_datetime(end))), ['date', column]]
    test_data.set_index('date', inplace=True)
    test_data.index.freq = 'D'
    return train_data, test_data

### Model ARIMA do predykcji wartości szeregu

In [ ]:
def fit_arima(country_name, column):
    x_train, x_test = get_train_test(country_name, column, '2020-06-29', '2022-07-04')
    check = adfuller(x_train)
    if check[1] > 0.05:
        x_train = x_train.diff().bfill()
        x_test = x_test.diff().bfill()
    model = ARIMA(x_train, order=(7, 1, 7)).fit()
    predition = np.round(model.forecast(len(x_test))).astype(np.int64)
    compare = pd.concat([predition, x_test], axis=1)
    compare.rename(columns={'predicted_mean': 'prediction', column: 'actual'}, inplace=True)
    return compare
        
def plot_arima(country_name, column, arima_data, ax=None):
    arima_data.plot(kind='line', ax=ax)
    if ax is None:
        plt.title(f'Skuteczność predykcji ARIMA dla kraju: {country_name}')
        plt.xlabel('Date')
        plt.ylabel(f'Wartości dla {column}')
    else:
        ax.set_title(f'Kraj: {country_name}')
        ax.set_xlabel('Date')
        ax.set_ylabel(f'Wartości dla {column}')
        
def get_arima_metrics(arima_data):
    return dict(zip(['MSE', 'MAE', 'R2'], [mean_squared_error(arima_data['actual'], arima_data['prediction']), mean_absolute_error(arima_data['actual'], arima_data['prediction']), r2_score(arima_data['actual'], arima_data['prediction'])]))

### Analiza szeregów czasowych dla 6 krajów wydających najwięcej na służbę zdrowia

In [ ]:
def time_series_analysis(column):
    fig0, ax0 = plt.subplots(nrows=3, ncols=2, figsize=(20, 12))
    fig1, ax1 = plt.subplots(nrows=len(countries), ncols=3, figsize=(20, 12))
    fig1.subplots_adjust(hspace=1.5)
    fig2, ax2 = plt.subplots(nrows=3, ncols=2, figsize=(20, 12))
    fig2.subplots_adjust(hspace=1.5)
    fig0.suptitle('Funkcja autokorelacji dla kraju', fontsize=20)
    fig1.suptitle('Dekompozycja sezonowa przy użyciu średnich kroczących', fontsize=20)
    fig2.suptitle(f'Wykorzystanie modelu ARIMA do predykcji wartości kolumny: {column}', fontsize=20)
    metrics = pd.DataFrame(columns=['MSE', 'MAE', 'R2'])
    for i, country_name in enumerate(countries):
        train, test = get_train_test(country_name, column, '2020-01-01', '2022-01-01', test_size=0.1)
        plot_acf(train, title=f'Autokorelacja dla kraju {country_name}', ax=ax0[divmod(i, 2)], zero=False)
        season_decomposition = seasonal_decompose(train, period=7)
        if i != len(countries) - 1:
            ax1[i, 0].set_ylabel('')
            ax1[i, 1].set_ylabel('')
            ax1[i, 2].set_ylabel('')
        season_decomposition.trend.plot(ax=ax1[i, 0], title=f'Trend dla {country_name}')  # długoterminowy ruch lub kierunek danych
        season_decomposition.resid.plot(ax=ax1[i, 1], title=f'Błąd dla {country_name}')  # zmienność danych, której nie można wytłumaczyć trendem lub sezonowością
        season_decomposition.seasonal.plot(ax=ax1[i, 2], title=f'Sezonowość dla {country_name}')  # wahania w regularnych odstępach czasu
        arima = fit_arima(country_name, column)
        plot_arima(country_name, column, arima, ax2[divmod(i, 2)])
        if metrics.empty:
            metrics = pd.DataFrame(get_arima_metrics(arima), columns=metrics.keys(), index=[country_name])
        else:
            metrics = pd.concat([metrics, pd.DataFrame(get_arima_metrics(arima), columns=metrics.keys(), index=[country_name])])
    display(metrics)

### 1.1 liczba nowych zachorowań,

In [ ]:
time_series_analysis('daily_confirmed')

### 1.2. liczby nowych śmierci,

In [ ]:
time_series_analysis('daily_deceased')

### 1.3. liczby nowych szczepień,

In [ ]:
time_series_analysis('daily_persons_vaccinated')

### 1.4 liczba nowych zachorowań mężczyzn,

In [ ]:
time_series_analysis('daily_confirmed_male')

### 1.4 liczba nowych zachorowań kobiet,

In [ ]:
time_series_analysis('daily_confirmed_female')

## <center>Cześć 3</center>
### Przygotuj dane treningowe w interesującym Cię okresie czasu (np. druga połowa 2020 roku i pierwsza połowa 2021 roku, tj. 52 tygodnie = X) oraz ewentualne dane testowe. Możesz przefiltrować dane także po innych kryteriach, jeżeli uznasz to za potrzebne. Zastosuj analizę regresji, aby przewidzieć wartości w kolejnych X tygodniach następujących po wybranym okresie treningowym. Użyj modelu regresji liniowej, gdzie zmienną niezależną będzie czas, a zmienną zależną:

### Model regresji liniowej/wielomianowej

In [ ]:
def linear_regression(train, test, ylabel, column, poly, ax=None):
    reg = LinearRegression(n_jobs=-1)
    map_train_dates = train.index.map(dt.datetime.toordinal)
    map_test_dates = test.index.map(dt.datetime.toordinal)
    if poly:
        polymonial = PolynomialFeatures(degree=5)
        poly_train = polymonial.fit_transform(pd.DataFrame(map_train_dates))
        poly_test = polymonial.transform(pd.DataFrame(map_test_dates))
        reg.fit(poly_train, train[column])
        y_pred = reg.predict(poly_test)
        sns.lineplot(x=train.index, y=reg.predict(poly_train).squeeze(), label='Predict', color='green', ax=ax)
    else:
        reg.fit(pd.DataFrame(map_train_dates), train[column])
        y_pred = reg.predict(pd.DataFrame(map_test_dates))
        sns.lineplot(x=train.index, y=reg.predict(pd.DataFrame(map_train_dates)).squeeze(), label='Predict', color='green', ax=ax)
    sns.scatterplot(x=train.index, y=train[column].squeeze(), label='Train', s=15, ax=ax)
    sns.scatterplot(x=test.index, y=test[column].squeeze(), label='Test', s=15, color='red', ax=ax)
    sns.lineplot(x=test.index, y=y_pred.squeeze(), color='green', ax=ax)
    metrics = dict(zip(['MSE', 'MAE', 'R2'], [mean_squared_error(test[column], y_pred), mean_absolute_error(test[column], y_pred), r2_score(test[column], y_pred)]))
    if ax is None:
        plt.legend()
        plt.xlabel('date')
        plt.ylabel(ylabel)
    else:
        ax.legend(loc='best')
        ax.set_xlabel('date')
        ax.set_ylabel(ylabel)
    return metrics

def plot_cool_plots(country_names, column, title, poly=False):
    fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(20, 12))
    fig.suptitle('Modele regresji liniowej', fontsize=20) if not poly else fig.suptitle('Modele regresji wielomianowej', fontsize=20)
    metrics = pd.DataFrame(columns=['MSE', 'MAE', 'R2'])
    for i, country_name in enumerate(country_names):
        X_train, X_test = get_train_test(country_name, column, '2020-06-29', '2021-07-04')
        reg_metrics = linear_regression(X_train, X_test, title, column, ax=ax[divmod(i, 2)], poly=poly)
        if metrics.empty:
            metrics = pd.DataFrame([reg_metrics], columns=reg_metrics.keys(), index=[country_name])
        else:
            metrics = pd.concat([metrics, pd.DataFrame([reg_metrics], columns=reg_metrics.keys(), index=[country_name])])
        ax[divmod(i, 2)].set_title(country_name)
    display(metrics)

### 3.1. liczba nowych zachorowań, dla Ameryki Północnej

In [ ]:
plot_cool_plots(['Canada', 'Mexico', 'United States of America', 'Cuba'], 'daily_confirmed', 'Liczba nowych zachorowań')

### 3.2. liczba nowych śmierci, dla krajów skandynawskich

In [ ]:
plot_cool_plots(['Norway', 'Sweden', 'Finland', 'Denmark'], 'daily_deceased', 'Liczba nowych śmierci')

### 3.3. liczba nowych szczepień, dla państw Bliskiego Wschodu

In [ ]:
plot_cool_plots(['Israel', 'Turkey', 'Iran', 'Saudi Arabia'], 'daily_persons_vaccinated', 'Liczba nowych szczepień')

### 3.4 liczba nowych zachorowań mężczyzn, dla krajów bałkańskich

In [ ]:
plot_cool_plots(['Albania', 'Bosnia and Herzegovina', 'Croatia', 'Serbia'], 'daily_confirmed_male', 'Liczba nowych zachorowań meżczyzn')

### 3.4 liczba nowych śmierci mężczyzn, dla krajów bałkańskich

In [ ]:
plot_cool_plots(['Albania', 'Bosnia and Herzegovina', 'Croatia', 'Serbia'], 'daily_deceased_male', 'Liczba nowych śmierci mężczyzn')

## <center>Część 4</center>
### Użyj model regresji wielomianowej dla wszystkich 5 przypadków rozważanych w części 3 poziomu 3. Oceń, który model regresji (liniowy czy wielomianowy) okazał się "lepszy" dla każdego przypadku. W tym celu możesz użyć dowolnej miary dopasowania modelu. Uzasadnij swoje zdanie.

### 4.1. liczba nowych zachorowań, dla Ameryki Północnej

In [ ]:
plot_cool_plots(['Canada', 'Mexico', 'United States of America', 'Cuba'], 'daily_confirmed', 'Liczba nowych zachorowań', poly=True)

### 4.2. liczba nowych śmierci, dla krajów skandynawskich

In [ ]:
plot_cool_plots(['Norway', 'Sweden', 'Finland', 'Denmark'], 'daily_deceased', 'Liczba nowych śmierci', poly=True)

### 4.3. liczba nowych szczepień, dla krajów Bliskiego Wschodu

In [ ]:
plot_cool_plots(['Israel', 'Turkey', 'Iran', 'Saudi Arabia'], 'daily_persons_vaccinated', 'Liczba nowych szczepień', poly=True)

### 4.4 liczba nowych zachorowań mężczyzn, dla krajów bałkańskich

In [ ]:
plot_cool_plots(['Albania', 'Bosnia and Herzegovina', 'Croatia', 'Serbia'], 'daily_confirmed_male', 'Liczba nowych zachorowań meżczyzn', poly=True)

### 4.4 liczba śmierci zachorowań mężczyzn, dla krajów bałkańskich

In [ ]:
plot_cool_plots(['Albania', 'Bosnia and Herzegovina', 'Croatia', 'Serbia'], 'daily_deceased_male', 'Liczba nowych śmierci mężczyzn', poly=True)